<a href="https://colab.research.google.com/github/Vishnu0920/BankManagementSystem/blob/main/GNNOnStudentDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cell 1
!pip install pandas torch scikit-learn
!pip install pyfm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.6/310.6 kB 2.5 MB/s eta 0:00:00
  Created wheel for pyfm: filename=pyfm-0.2.4-py3-none-any.whl size=13207 sha256=c5f8a61ef44af6a6646221a3e15e46ef90e653f8d1c212ec0a5d26c9899cf68d
  Stored in directory: /root/.cache/pip/wheels/fb/5e/81/2218255b6118f1373b7b77b2062d8d2e7f757186cedca7becc
Successfully built pyfm


In [ ]:
# Cell 2: Import libraries
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Cell 3: Load the dataset
df = pd.read_csv('grade_data.csv')

In [ ]:
# Cell 4: Encode student_id and course_id using LabelEncoder
le_student = LabelEncoder()
le_course = LabelEncoder()

df['student_id'] = le_student.fit_transform(df['student_id'])
df['course_id'] = le_course.fit_transform(df['course_id'])

In [ ]:
# Cell 5: Map course grades to the range [0, 1] for regression
df['course_grade'] = df['course_grade'] / 10.0

In [ ]:
# Cell 6: Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Cell 7: Convert the data to PyTorch tensors
train_user = torch.LongTensor(train_df['student_id'].values)
train_course = torch.LongTensor(train_df['course_id'].values)
train_grade = torch.FloatTensor(train_df['course_grade'].values)

test_user = torch.LongTensor(test_df['student_id'].values)
test_course = torch.LongTensor(test_df['course_id'].values)
test_grade = torch.FloatTensor(test_df['course_grade'].values)


In [ ]:
class NGCF(nn.Module):
    def __init__(self, num_users, num_courses, embed_size=64):
        super(NGCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embed_size)
        self.course_embedding = nn.Embedding(num_courses, embed_size)
        self.fc1 = nn.Linear(embed_size * 2, embed_size)  # Corrected the input size
        self.fc2 = nn.Linear(embed_size, embed_size)

    def forward(self, user, course):
        user_embed = self.user_embedding(user)
        course_embed = self.course_embedding(course)

        # Concatenate user and course embeddings
        combined = torch.cat([user_embed, course_embed], dim=1)

        # Apply fully connected layers
        combined = self.fc1(combined)
        combined = nn.ReLU()(combined)
        combined = self.fc2(combined)

        return combined


In [ ]:
# Cell 9: Hyperparameters
num_users = len(le_student.classes_)
num_courses = len(le_course.classes_)
embedding_dim = 64
num_epochs = 10
batch_size = 64
learning_rate = 0.001


In [ ]:
# Cell 10: Create DataLoader for training
train_dataset = TensorDataset(train_user, train_course, train_grade)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Cell 11: Instantiate the model, define loss function and optimizer
model = NGCF(num_users, num_courses, embedding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set the print interval
print_interval = 10  # Adjust the interval as needed


In [ ]:
# Cell 12: Training loop with performance monitoring
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch_index, batch in enumerate(train_dataloader):
        user, course, grade = batch
        optimizer.zero_grad()

        output = model(user, course).squeeze()

        # Ensure output and grade have the same shape
        grade = grade.view(-1, 1)  # Reshape grade to (batch_size, 1) to match output shape

        loss = criterion(output, grade)
        loss.backward()
        optimizer.step()

        # Print training loss at regular intervals
        if batch_index % print_interval == 0:
            print(f'Epoch {epoch}, Batch {batch_index}, Loss: {loss.item()}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Batch 0, Loss: 0.6770484447479248
Epoch 0, Batch 10, Loss: 0.4091627597808838
Epoch 0, Batch 20, Loss: 0.2490861415863037
Epoch 0, Batch 30, Loss: 0.15160611271858215
Epoch 0, Batch 40, Loss: 0.10299594700336456
Epoch 0, Batch 50, Loss: 0.08319690823554993
Epoch 0, Batch 60, Loss: 0.06753396987915039
Epoch 0, Batch 70, Loss: 0.05856309458613396
Epoch 0, Batch 80, Loss: 0.054099924862384796
Epoch 0, Batch 90, Loss: 0.0620177760720253
Epoch 0, Batch 100, Loss: 0.05191892758011818
Epoch 0, Batch 110, Loss: 0.04519454017281532
Epoch 0, Batch 120, Loss: 0.04784328490495682
Epoch 0, Batch 130, Loss: 0.048547353595495224
Epoch 0, Batch 140, Loss: 0.0551079586148262
Epoch 0, Batch 150, Loss: 0.03803477808833122
Epoch 0, Batch 160, Loss: 0.03584242984652519
Epoch 0, Batch 170, Loss: 0.044491082429885864
Epoch 0, Batch 180, Loss: 0.04091908782720566
Epoch 0, Batch 190, Loss: 0.03533342853188515
Epoch 0, Batch 200, Loss: 0.04141280800104141
Epoch 0, Batch 210, Loss: 0.04246652126312256
E

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([42, 1])) that is different to the input size (torch.Size([42, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Batch 50, Loss: 0.02472699247300625
Epoch 1, Batch 60, Loss: 0.024567048996686935
Epoch 1, Batch 70, Loss: 0.024860592558979988
Epoch 1, Batch 80, Loss: 0.025089504197239876
Epoch 1, Batch 90, Loss: 0.01617947220802307
Epoch 1, Batch 100, Loss: 0.019884582608938217
Epoch 1, Batch 110, Loss: 0.02272436022758484
Epoch 1, Batch 120, Loss: 0.016819240525364876
Epoch 1, Batch 130, Loss: 0.017689619213342667
Epoch 1, Batch 140, Loss: 0.01152730081230402
Epoch 1, Batch 150, Loss: 0.01621892675757408
Epoch 1, Batch 160, Loss: 0.017710402607917786
Epoch 1, Batch 170, Loss: 0.01603238843381405
Epoch 1, Batch 180, Loss: 0.01740460842847824
Epoch 1, Batch 190, Loss: 0.02030702866613865
Epoch 1, Batch 200, Loss: 0.01678253337740898
Epoch 1, Batch 210, Loss: 0.019896535202860832
Epoch 1, Batch 220, Loss: 0.016224846243858337
Epoch 1, Batch 230, Loss: 0.02499503456056118
Epoch 1, Batch 240, Loss: 0.02150282822549343
Epoch 1, Batch 250, Loss: 0.020903846248984337
Epoch 1, Batch 260, Loss: 0.0

In [ ]:
# Cell 13: Evaluation on the test dataset
model.eval()  # Set the model to evaluation mode
total_test_loss = 0.0
num_test_samples = 0

# Evaluate the model on the test dataset
with torch.no_grad():
    for batch in test_dataloader:
        user, course, grade = batch
        output = model(user, course).squeeze()

        # Ensure output and grade have the same shape
        grade = grade.view(-1, 1)  # Reshape grade to (batch_size, 1) to match output shape

        loss = criterion(output, grade)
        total_test_loss += loss.item()
        num_test_samples += len(grade)

# Calculate Mean Squared Error (MSE) on the test dataset
test_mse = total_test_loss / num_test_samples

print(f'Mean Squared Error (MSE) on the test dataset: {test_mse}')


Mean Squared Error (MSE) on the test dataset: 0.00023545690095159115


In [ ]:
def recommend_courses(model, user_and_courses, top_k):
    input_user, input_courses = user_and_courses

    # Convert input_user to PyTorch tensor
    user_ids = torch.LongTensor([input_user])

    # Generate all possible course IDs
    all_course_ids = torch.arange(len(le_course.classes_))

    # Repeat the given user_id for all courses
    user_ids = torch.full_like(all_course_ids, fill_value=user_ids[0])

    # Make predictions for all courses for the given student
    predictions = model(user_ids, all_course_ids).squeeze()

    # Get the indices of the top-k predictions
    num_recommendations = min(top_k, len(predictions))
    top_indices = torch.topk(predictions, num_recommendations).indices

    # Reshape top_indices to 1D tensor
    top_indices = top_indices.view(-1)

    # Map the top indices back to the course IDs
    top_course_ids = le_course.inverse_transform(top_indices.numpy())

    # Exclude courses already in input_courses from recommendations
    recommended_courses = []
    unique_course_ids = set()
    for course_id in top_course_ids:
        if course_id not in input_courses and course_id not in unique_course_ids:
            recommended_courses.append(course_id)
            unique_course_ids.add(course_id)
            if len(recommended_courses) == top_k:
                break

    return recommended_courses


In [ ]:
# Example: Input student's grade for previous courses
input_user = 123  # Replace with the actual student ID

# Use the actual course labels seen during training
input_courses = {
    'CHEMISTRY LABORATORY': 8,
    'GENERAL CHEMISTRY': 7,
    'ELECTRICAL SCIENCES': 3,
    'ADDITIVE MANUFACTURING': 1,
    'PRACTICE SCHOOL I':10,
    'PHYSICS LABORATORY':5
    # Add more courses and grades as needed
}

# Set top_k to the desired number
top_k = 5

# Call the function with the updated course grades
user_and_courses = (input_user, input_courses)
recommended_courses = recommend_courses(model, user_and_courses, top_k)

print(f"Top {top_k} recommended courses for the student based on previous grades: {recommended_courses}")


Top 5 recommended courses for the student based on previous grades: ['APPLIED STOCHASTIC PROCESS', 'ARTIFICIAL INTELLIGENCE FOR RO', 'CINEMATIC ADAPTATION', 'ADV TRANSPORT PHENOMENA', 'ANALOG & DIGIT VLSI DES']
